In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Fits to Data with Constrained D/S Ratio
This notebook covers analysis of the pwa results, for fits to $\omega\pi^0$ data. The waveset / data used can be summarized with:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
* The D and S waves are constrained to each other, such that every $D$-wave has a common ratio and phase between its $S$-wave partner
  * Boundaries: $0< r_{D/S} < 1,\quad -\pi<\phi_{D-S}<\pi$
* Include an isotropic background term
  * Assumed that sideband subtraction handles almost all background events, and so these events will be *true* $\omega$ events, but contribute in total a flat distribution to the angular distributions
* A cut placed on the mass of the proton-pseudoscalar system $M_{p\pi^0} > 1.4$ GeV is imposed to remove $\Delta^+$ contributions in the low mass range 
* Uses all 4 polarization orientations

The details of the fit can be described as:
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 2.0$ 
  * Varying bins of $-t$: [0.1, 0.2], [0.2, 0.3], [0.3, 0.5], [0.5, 1.0]
* Perform 500 randomized fits in each bin of mass and $-t$ independently 
* Use the best of the 500 fits to seed 100 bootstrap fits

Finer details can be found in the print-out of the `.yaml` file below

## Setup


In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict
import warnings

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1-constrained-ratio/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

The fit results have all been collected, saved to dataframes, and preprocessed using the [`scripts/run_analysis.py`](https://github.com/kevScheuer/neutralb1/blob/main/scripts/run_analysis.py) module. We can simply load in the pickled files here

In [ ]:
# load in the preprocessed results 
results = {} # "detected" results
ac_results = {} # acceptance corrected results

for t_bin in sorted(os.listdir(f"{STUDY_DIR}/")):
    if not os.path.isdir(f"{STUDY_DIR}/{t_bin}"):
        continue
    print(f"Loading results from {t_bin}")
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results.pkl", "rb") as f:
        data = pkl.load(f)
    results[t_bin] = ResultManager(**data)

    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
    ac_results[t_bin] = ResultManager(**data)

# let interpreter know the types explicitly so we can easily access methods
results: Dict[str, ResultManager]
ac_results: Dict[str, ResultManager]

## Analysis

### Standard Plots
First step is to print out all the standard plots to view the model's performance

In [ ]:
for t_bin in results.keys():
    utils.big_print(f"t bin: {t_bin}", 2.0)
    for pdf in pathlib.Path(f"{STUDY_DIR}/{t_bin}/plots").glob("*.pdf"):
            if pdf.name in ["jp.pdf", "waves.pdf", "matrix.pdf", "proj_moments.pdf"]:
                utils.display_pdf(str(pdf), 0)

### D/S Ratio
Compare the D/S Ratio to E852 results, and check for correlations using the bootstrap fits

In [ ]:
plt.rcParams["figure.dpi"] = 300
results["t_0.1-0.2"].plot.diagnostic.ds_ratio()

### b1 - Vector Interaction

In [ ]:
results["t_0.1-0.2"].plot.phase.mass_phase("p1p0S", "p1mpP", {"color": "tab:blue"}, {"color": "tab:orange"})

### Naturality Dominance
Use the acceptance corrected data to view the reflectivity contributions, and thus the naturality of our exchange particle

In [ ]:
fig, axs = plt.subplots(
    2,
    1,    
    sharey="row",
    sharex=True,
    figsize=(8, 7),
    layout="constrained",
    squeeze=False
)

# Store handles and labels for the legend
handles = []
labels = []
bin_width = None

for i, t_bin in enumerate(ac_results.keys()):
    result = ac_results[t_bin]
    masses = result.plot.intensity._masses
    bin_width = result.plot.intensity._bin_width

    # ##### 1P COHERENT SUM #####  
    
    h1 = axs[0, i].errorbar(
        masses, 
        result.fit_df["p1p"],
        result.plot.intensity.get_bootstrap_error("p1p"),
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="darkred",
        label=rf"$1^+$ (Natural)",
    )
    h2 = axs[0, i].errorbar(
        masses, 
        result.fit_df["p1m"],
        result.plot.intensity.get_bootstrap_error("p1m"),
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="lightcoral",
        label=rf"$1^-$ (Natural)",
    )

    # ##### 1M COHERENT SUM #####
    
    h3 = axs[1, i].errorbar(
        masses, 
        result.fit_df["m1p"],
        result.plot.intensity.get_bootstrap_error("m1p"),
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="darkblue",
        label=rf"$1^+$ (Unnatural)",
    )
    h4 = axs[1, i].errorbar(
        masses, 
        result.fit_df["m1m"],
        result.plot.intensity.get_bootstrap_error("m1m"),
        bin_width / 2,
        marker="",
        linestyle="-",
        markersize=6,
        color="lightblue",
        label=rf"$1^-$ (Unnatural)",
    )

    # Collect legend handles and labels only from the first subplot to avoid duplicates
    if i == 0:
        handles.extend([h1, h2, h3, h4])
        labels.extend([h.get_label() for h in [h1, h2, h3, h4]])

    low_t, high_t = t_bin.split('_')[1].split('-')
    axs[0, i].set_title(rf" {low_t} < $-t$ <  {high_t} GeV$^2$", pad=10, fontsize=16)
    axs[0, i].set_ylim(bottom=0.0)
    axs[1, i].set_ylim(bottom=0.0)
    axs[1, i].set_xlabel(rf"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
axs[1, 0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

# Create a single legend for the entire figure positioned on the right
fig.legend(handles, labels, loc='center right', bbox_to_anchor=(1.33, 0.5))

### Bootstrap Distributions at Discontinuity

In [ ]:
# find problem fit indices
results["t_0.1-0.2"].data_df[results["t_0.1-0.2"].data_df["m_high"] == 1.400]["fit_index"].values[0]